In [13]:
import sys
import re
import numpy as np
import ast

In [14]:
sys.path.append('MLSTM-FCN/')

In [15]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, multiply, concatenate, Activation, Masking, Reshape
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout

from utils.constants import MAX_NB_VARIABLES, NB_CLASSES_LIST, MAX_TIMESTEPS_LIST
from utils.keras_utils import train_model, evaluate_model, set_trainable
from utils.layer_utils import AttentionLSTM

In [16]:
resultsFilename='results_data_FINAL.txt'

In [17]:
DATASET_INDEX_0 = 56
DATASET_INDEX_1 = 57
DATASET_INDEX_2 = 58
DATASET_INDEX_3 = 59

MAX_TIMESTEPS = MAX_TIMESTEPS_LIST[DATASET_INDEX_0]
MAX_NB_VARIABLES = MAX_NB_VARIABLES[DATASET_INDEX_0]
NB_CLASS = NB_CLASSES_LIST[DATASET_INDEX_0]

TRAINABLE = True

In [19]:
def generate_model_2():
    ip = Input(shape=(MAX_NB_VARIABLES, MAX_TIMESTEPS))
    # stride = 10

    # x = Permute((2, 1))(ip)
    # x = Conv1D(MAX_NB_VARIABLES // stride, 8, strides=stride, padding='same', activation='relu', use_bias=False,
    #            kernel_initializer='he_uniform')(x)  # (None, variables / stride, timesteps)
    # x = Permute((2, 1))(x)

    #ip1 = K.reshape(ip,shape=(MAX_TIMESTEPS,MAX_NB_VARIABLES))
    #x = Permute((2, 1))(ip)
    x = Masking()(ip)
    x = AttentionLSTM(8)(x)
    x = Dropout(0.8)(x)

    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(NB_CLASS, activation='softmax')(x)

    model = Model(ip, out)
    model.summary()

    # add load model code here to fine-tune

    return model

In [20]:
def generate_model_3():
    ip = Input(shape=(MAX_NB_VARIABLES, MAX_TIMESTEPS))

    x = Masking()(ip)
    x = LSTM(8)(x)
    x = Dropout(0.8)(x)

    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    #y = squeeze_excite_block(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    #y = squeeze_excite_block(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(NB_CLASS, activation='softmax')(x)

    model = Model(ip, out)
    model.summary()

    # add load model code here to fine-tune

    return model

In [21]:
def generate_model_4():
    ip = Input(shape=(MAX_NB_VARIABLES, MAX_TIMESTEPS))
    # stride = 3
    #
    # x = Permute((2, 1))(ip)
    # x = Conv1D(MAX_NB_VARIABLES // stride, 8, strides=stride, padding='same', activation='relu', use_bias=False,
    #            kernel_initializer='he_uniform')(x)  # (None, variables / stride, timesteps)
    # x = Permute((2, 1))(x)

    x = Masking()(ip)
    x = AttentionLSTM(8)(x)
    x = Dropout(0.8)(x)

    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    #y = squeeze_excite_block(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    #y = squeeze_excite_block(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(NB_CLASS, activation='softmax')(x)

    model = Model(ip, out)
    model.summary()

    # add load model code here to fine-tune

    return model

In [22]:
def squeeze_excite_block(input):
    ''' Create a squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor
    Returns: a keras tensor
    '''
    filters = input._keras_shape[-1] # channel_axis = -1 for TF

    se = GlobalAveragePooling1D()(input)
    se = Reshape((1, filters))(se)
    se = Dense(filters // 16,  activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    se = multiply([input, se])
    return se

In [23]:
n_epochs=250

In [24]:
if __name__ == "__main__":
    with open(resultsFilename, "a") as text_file:
        print(f"Current Fold: 0", file=text_file)
    model = generate_model_2()
    train_model(model, DATASET_INDEX_0, dataset_prefix='eeg2_attention', epochs=n_epochs, batch_size=128)
    evaluate_model(model, DATASET_INDEX_0, dataset_prefix='eeg2_attention', batch_size=128)
    with open(resultsFilename, "a") as text_file:
        print(f"Current Fold: 1", file=text_file)
    model = generate_model_2()
    train_model(model, DATASET_INDEX_1, dataset_prefix='eeg2_attention', epochs=n_epochs, batch_size=128)
    evaluate_model(model, DATASET_INDEX_1, dataset_prefix='eeg2_attention', batch_size=128)
    with open(resultsFilename, "a") as text_file:
        print(f"Current Fold: 2", file=text_file)
    model = generate_model_2()
    train_model(model, DATASET_INDEX_2, dataset_prefix='eeg2_attention', epochs=n_epochs, batch_size=128)
    evaluate_model(model, DATASET_INDEX_2, dataset_prefix='eeg2_attention', batch_size=128)
    with open(resultsFilename, "a") as text_file:
        print(f"Current Fold: 3", file=text_file)
    model = generate_model_2()
    train_model(model, DATASET_INDEX_3, dataset_prefix='eeg2_attention', epochs=n_epochs, batch_size=128)
    evaluate_model(model, DATASET_INDEX_3, dataset_prefix='eeg2_attention', batch_size=128)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 16, 7680)     0                                            
__________________________________________________________________________________________________
permute_2 (Permute)             (None, 7680, 16)     0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 7680, 128)    16512       permute_2[0][0]                  
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 7680, 128)    512         conv1d_4[0][0]                   
__________________________________________________________________________________________________
activation

 - 16s - loss: 0.4595 - acc: 0.7581 - f1_m: 0.7581 - precision_m: 0.7581 - recall_m: 0.7581 - tp_m: 47.0000 - fp_m: 15.0000 - tn_m: 47.0000 - fn_m: 15.0000 - val_loss: 0.6321 - val_acc: 0.6471 - val_f1_m: 0.6471 - val_precision_m: 0.6471 - val_recall_m: 0.6471 - val_tp_m: 11.0000 - val_fp_m: 6.0000 - val_tn_m: 11.0000 - val_fn_m: 6.0000

Epoch 00004: loss improved from 0.50057 to 0.45947, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 16.202544450759888
Epoch 5/250
epoch time start: 1578287161.8240325
 - 16s - loss: 0.4409 - acc: 0.7903 - f1_m: 0.7903 - precision_m: 0.7903 - recall_m: 0.7903 - tp_m: 49.0000 - fp_m: 13.0000 - tn_m: 49.0000 - fn_m: 13.0000 - val_loss: 0.5708 - val_acc: 0.5882 - val_f1_m: 0.5882 - val_precision_m: 0.5882 - val_recall_m: 0.5882 - val_tp_m: 10.0000 - val_fp_m: 7.0000 - val_tn_m: 10.0000 - val_fn_m: 7.0000

Epoch 00005: loss improved from 0.45947 to 0.44091, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 16


Epoch 00020: loss improved from 0.10832 to 0.08976, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 16.182763814926147
Epoch 21/250
epoch time start: 1578287421.2241862
 - 16s - loss: 0.0856 - acc: 0.9839 - f1_m: 0.9839 - precision_m: 0.9839 - recall_m: 0.9839 - tp_m: 61.0000 - fp_m: 1.0000 - tn_m: 61.0000 - fn_m: 1.0000 - val_loss: 0.1995 - val_acc: 0.9412 - val_f1_m: 0.9412 - val_precision_m: 0.9412 - val_recall_m: 0.9412 - val_tp_m: 16.0000 - val_fp_m: 1.0000 - val_tn_m: 16.0000 - val_fn_m: 1.0000

Epoch 00021: loss improved from 0.08976 to 0.08562, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 16.1155743598938
Epoch 22/250
epoch time start: 1578287437.3399003
 - 16s - loss: 0.0680 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.1891 - val_acc: 0.9412 - val_f1_m: 0.9412 - val_precision_m: 0.9412 - val_recall_m: 0.9412 - val_tp


Epoch 00036: loss improved from 0.01277 to 0.01009, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 16.282090425491333
Epoch 37/250
epoch time start: 1578287680.3118308
 - 16s - loss: 0.0092 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.1408 - val_acc: 0.9412 - val_f1_m: 0.9412 - val_precision_m: 0.9412 - val_recall_m: 0.9412 - val_tp_m: 16.0000 - val_fp_m: 1.0000 - val_tn_m: 16.0000 - val_fn_m: 1.0000

Epoch 00037: loss improved from 0.01009 to 0.00915, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 16.095375537872314
Epoch 38/250
epoch time start: 1578287696.4073303
 - 16s - loss: 0.0096 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.1364 - val_acc: 0.9412 - val_f1_m: 0.9412 - val_precision_m: 0.9412 - val_recall_m: 0.941


Epoch 00052: loss did not improve
epoch time measured: 16.13459014892578
Epoch 53/250
epoch time start: 1578287938.9026732
 - 16s - loss: 0.0016 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0893 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_m: 17.0000 - val_fn_m: 0.0000e+00

Epoch 00053: loss improved from 0.00163 to 0.00160, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 16.23422861099243
Epoch 54/250
epoch time start: 1578287955.1370566
 - 16s - loss: 0.0015 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0874 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_m: 17.0000

...


Epoch 00170: loss did not improve
epoch time measured: 16.07416272163391
Epoch 171/250
epoch time start: 1578289845.8744574
 - 16s - loss: 2.0057e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0176 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_m: 17.0000 - val_fn_m: 0.0000e+00

Epoch 00171: loss did not improve
epoch time measured: 15.997064590454102
Epoch 172/250
epoch time start: 1578289861.8716671
 - 16s - loss: 1.9092e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0176 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_m: 17.0000 - val_fn_m: 0.0000e+00

Epoch 00172: loss did not improve



Epoch 00187: loss did not improve
epoch time measured: 16.146645307540894
Epoch 188/250
epoch time start: 1578290119.7440858
 - 16s - loss: 2.1625e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0172 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_m: 17.0000 - val_fn_m: 0.0000e+00

Epoch 00188: loss did not improve
epoch time measured: 16.00996494293213
Epoch 189/250
epoch time start: 1578290135.754365
 - 16s - loss: 1.7865e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0176 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_m: 17.0000 - val_fn_m: 0.0000e+00

Epoch 00189: loss improved from 0.0


Epoch 00204: loss did not improve
epoch time measured: 16.03717827796936
Epoch 205/250
epoch time start: 1578290393.6616251
 - 16s - loss: 1.6802e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0164 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_m: 17.0000 - val_fn_m: 0.0000e+00

Epoch 00205: loss did not improve
epoch time measured: 16.105181217193604
Epoch 206/250
epoch time start: 1578290409.7669356
 - 16s - loss: 1.8295e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0171 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_m: 17.0000 - val_fn_m: 0.0000e+00

Epoch 00206: loss did not improve



Epoch 00221: loss did not improve
epoch time measured: 16.06287670135498
Epoch 222/250
epoch time start: 1578290668.3539407
 - 16s - loss: 1.5262e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0146 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_m: 17.0000 - val_fn_m: 0.0000e+00

Epoch 00222: loss did not improve
epoch time measured: 16.27842378616333
Epoch 223/250
epoch time start: 1578290684.6324937
 - 16s - loss: 1.5024e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0143 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_m: 17.0000 - val_fn_m: 0.0000e+00

Epoch 00223: loss improved from 0.0


Epoch 00238: loss did not improve
epoch time measured: 16.04261350631714
Epoch 239/250
epoch time start: 1578290942.7668355
 - 16s - loss: 1.3160e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0161 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_m: 17.0000 - val_fn_m: 0.0000e+00

Epoch 00239: loss improved from 0.00014 to 0.00013, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 16.100348711013794
Epoch 240/250
epoch time start: 1578290958.867333
 - 16s - loss: 1.4884e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0158 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 16, 7680)     0                                            
__________________________________________________________________________________________________
permute_1 (Permute)             (None, 7680, 16)     0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 7680, 128)    16512       permute_1[0][0]                  
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 7680, 128)    512         conv1d_1[0][0]                   
__________________________________________________________________________________________________
activation

epoch time start: 1578261866.8861992
Epoch 5/250

Epoch 00005: loss improved from 0.45860 to 0.45542, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 17.539481163024902
 - 18s - loss: 0.4554 - acc: 0.7742 - f1_m: 0.7742 - precision_m: 0.7742 - recall_m: 0.7742 - tp_m: 48.0000 - fp_m: 14.0000 - tn_m: 48.0000 - fn_m: 14.0000 - val_loss: 0.5851 - val_acc: 0.5294 - val_f1_m: 0.5294 - val_precision_m: 0.5294 - val_recall_m: 0.5294 - val_tp_m: 9.0000 - val_fp_m: 8.0000 - val_tn_m: 9.0000 - val_fn_m: 8.0000
epoch time start: 1578261884.4271972
Epoch 6/250

Epoch 00006: loss improved from 0.45542 to 0.38435, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 18.56748366355896
 - 19s - loss: 0.3844 - acc: 0.8548 - f1_m: 0.8548 - precision_m: 0.8548 - recall_m: 0.8548 - tp_m: 53.0000 - fp_m: 9.0000 - tn_m: 53.0000 - fn_m: 9.0000 - val_loss: 0.4296 - val_acc: 0.9412 - val_f1_m: 0.9412 - val_precision_m: 0.9412 - val_recall_m: 0.9412 - val_tp_m: 16.00

epoch time start: 1578262151.2273283
Epoch 21/250

Epoch 00021: loss improved from 0.11339 to 0.10737, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 17.382821321487427
 - 17s - loss: 0.1074 - acc: 0.9839 - f1_m: 0.9839 - precision_m: 0.9839 - recall_m: 0.9839 - tp_m: 61.0000 - fp_m: 1.0000 - tn_m: 61.0000 - fn_m: 1.0000 - val_loss: 0.4459 - val_acc: 0.8235 - val_f1_m: 0.8235 - val_precision_m: 0.8235 - val_recall_m: 0.8235 - val_tp_m: 14.0000 - val_fp_m: 3.0000 - val_tn_m: 14.0000 - val_fn_m: 3.0000
epoch time start: 1578262168.6119695
Epoch 22/250

Epoch 00022: loss improved from 0.10737 to 0.09196, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 17.490198612213135
 - 17s - loss: 0.0920 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.3652 - val_acc: 0.8235 - val_f1_m: 0.8235 - val_precision_m: 0.8235 - val_recall_m: 0.8235 - val_

epoch time start: 1578262433.442777
Epoch 37/250

Epoch 00037: loss improved from 0.02127 to 0.01986, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 17.374009609222412
 - 17s - loss: 0.0199 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.2102 - val_acc: 0.9412 - val_f1_m: 0.9412 - val_precision_m: 0.9412 - val_recall_m: 0.9412 - val_tp_m: 16.0000 - val_fp_m: 1.0000 - val_tn_m: 16.0000 - val_fn_m: 1.0000
epoch time start: 1578262450.8183672
Epoch 38/250

Epoch 00038: loss improved from 0.01986 to 0.01659, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 17.479416131973267
 - 17s - loss: 0.0166 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.1944 - val_acc: 0.9412 - val_f1_m: 0.9412 - val_precision_m: 0.9412 - val_recall_m: 0.9412

epoch time start: 1578262716.573362
Epoch 53/250

Epoch 00053: loss improved from 0.00263 to 0.00213, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 17.796303749084473
 - 18s - loss: 0.0021 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.2444 - val_acc: 0.9412 - val_f1_m: 0.9412 - val_precision_m: 0.9412 - val_recall_m: 0.9412 - val_tp_m: 16.0000 - val_fp_m: 1.0000 - val_tn_m: 16.0000 - val_fn_m: 1.0000
epoch time start: 1578262734.3712823
Epoch 54/250

Epoch 00054: loss improved from 0.00213 to 0.00175, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 17.639718055725098
 - 18s - loss: 0.0018 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.2601 - val_acc: 0.8824 - val_f1_m: 0.8824 - val_precision_m: 0.8824 - val_recall_m: 0.8824

...

epoch time start: 1578267656.4374526
Epoch 83/250

Epoch 00083: loss did not improve
epoch time measured: 17.80473232269287
 - 18s - loss: 0.0011 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0700 - val_acc: 0.9412 - val_f1_m: 0.9412 - val_precision_m: 0.9412 - val_recall_m: 0.9412 - val_tp_m: 16.0000 - val_fp_m: 1.0000 - val_tn_m: 16.0000 - val_fn_m: 1.0000
epoch time start: 1578267674.24367
Epoch 84/250

Epoch 00084: loss improved from 0.00102 to 0.00102, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 17.743468761444092
 - 18s - loss: 0.0010 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0698 - val_acc: 0.9412 - val_f1_m: 0.9412 - val_precision_m: 0.9412 - val_recall_m: 0.9412 - val_tp_m: 16.0000 - val_fp_m: 1.0000 - val_tn_m: 16.0000 - val_fn_m: 

epoch time start: 1578267957.0149858
Epoch 100/250

Epoch 00100: loss did not improve
epoch time measured: 17.648056983947754
 - 18s - loss: 8.5633e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0690 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_m: 17.0000 - val_fn_m: 0.0000e+00
epoch time start: 1578267974.664178
Epoch 101/250

Epoch 00101: loss did not improve
epoch time measured: 17.2295823097229
 - 17s - loss: 7.9052e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0696 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_m: 17.0000 - val_fn_m: 0.0000e+00
epoch time start: 1578267991.894799
E

epoch time start: 1578268255.2799776
Epoch 117/250

Epoch 00117: loss did not improve
epoch time measured: 17.23207139968872
 - 17s - loss: 6.3711e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0731 - val_acc: 0.9412 - val_f1_m: 0.9412 - val_precision_m: 0.9412 - val_recall_m: 0.9412 - val_tp_m: 16.0000 - val_fp_m: 1.0000 - val_tn_m: 16.0000 - val_fn_m: 1.0000
epoch time start: 1578268272.5131276
Epoch 118/250

Epoch 00118: loss did not improve
epoch time measured: 17.730239152908325
 - 18s - loss: 6.1776e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0736 - val_acc: 0.9412 - val_f1_m: 0.9412 - val_precision_m: 0.9412 - val_recall_m: 0.9412 - val_tp_m: 16.0000 - val_fp_m: 1.0000 - val_tn_m: 16.0000 - val_fn_m: 1.0000
epoch time start: 1578268290.2444541
Epoch 119/250


epoch time start: 1578268555.2667723
Epoch 134/250

Epoch 00134: loss did not improve
epoch time measured: 17.703036308288574
 - 18s - loss: 5.4789e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0638 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_m: 17.0000 - val_fn_m: 0.0000e+00
epoch time start: 1578268572.9710224
Epoch 135/250

Epoch 00135: loss did not improve
epoch time measured: 17.48184633255005
 - 17s - loss: 5.5179e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0631 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_m: 17.0000 - val_fn_m: 0.0000e+00
epoch time start: 1578268590.454021

epoch time start: 1578268854.1287782
Epoch 151/250

Epoch 00151: loss did not improve
epoch time measured: 17.464759349822998
 - 17s - loss: 4.1567e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0575 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_m: 17.0000 - val_fn_m: 0.0000e+00
epoch time start: 1578268871.5949068
Epoch 152/250

Epoch 00152: loss did not improve
epoch time measured: 17.501006364822388
 - 18s - loss: 4.4030e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 62.0000 - fp_m: 0.0000e+00 - tn_m: 62.0000 - fn_m: 0.0000e+00 - val_loss: 0.0574 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000 - val_tp_m: 17.0000 - val_fp_m: 0.0000e+00 - val_tn_m: 17.0000 - val_fn_m: 0.0000e+00
epoch time start: 1578268889.09707

In [ ]:
filepath = 'results_data_FINAL.txt'

In [ ]:
f = open(filepath)
accList=list()
precisionList=list()
recallList=list()
f1List=list()

for line in f:
    if(str(line[0:13])=='Fold Accuracy'):
        listStart1=line.find('[')
        listEnd1=line.find(']')
        list1=ast.literal_eval(line[listStart1:listEnd1+1])
        accList.append(list1)
    if(str(line[0:14])=='Fold Precision'):
        listStart2=line.find('[')
        listEnd2=line.find(']')
        list2=ast.literal_eval(line[listStart2:listEnd2+1])
        precisionList.append(list2)
    if(str(line[0:11])=='Fold Recall'):
        listStart3=line.find('[')
        listEnd3=line.find(']')
        list3=ast.literal_eval(line[listStart3:listEnd3+1])
        recallList.append(list3)
    if(str(line[0:7])=='Fold F1'):
        listStart4=line.find('[')
        listEnd4=line.find(']')
        list4=ast.literal_eval(line[listStart4:listEnd4+1])
        f1List.append(list4)
f.close()

In [ ]:
allAccuracies=np.array(accList)
averageAccuracy=np.average(allAccuracies, axis=0)

In [ ]:
allPrecisions=np.array(precisionList)
averagePrecisions=np.average(allPrecisions, axis=0)

In [ ]:
allRecall=np.array(recallList)
averageRecall=np.average(allRecall, axis=0)

In [ ]:
allF1s=np.array(f1List)
averageF1s=np.average(allF1s, axis=0)

In [ ]:
averageAccuracy

In [ ]:
averagePrecisions

In [ ]:
averageRecall

In [ ]:
averageF1s

In [ ]:
with open(resultsFilename, "a") as text_file:
    print(f"Average Accuracies List: {str(averageAccuracy)}", file=text_file)
    print(f"Average Precisions List: {str(averagePrecisions)}", file=text_file)
    print(f"Average Recalls List: {str(averageRecall)}", file=text_file)
    print(f"Average F1 List: {str(averageF1s)}", file=text_file)

